In [26]:
N = 11
p = 3
q = 2^5
Zx.<x> = ZZ[]
T = Zx.change_ring(Integers(p)).quotient(x^N-1)
f = -1 + x + x^2 - x^4 + x^6 + x^9 - x^10
g = -1 + x^2 + x^3 + x^5 - x^8 - x^10

In [2]:
def balancedmod(f,q):
    g = list(((f[i] + q//2) % q) - q//2 for i in range(N))
    Zx.<x> = ZZ[]
    return Zx(g)

In [3]:
def convolution(f,g):
    return (f * g) % (x^N-1)

In [4]:
def findDegree(coefs_list):
    len_ = len(coefs_list)
    for i in range(len_ - 1, -1, -1):
        if coefs_list[i] != 0:
            return i

def compareDegrees(coefs_list1, coefs_list2):
    c1 = 0
    c2 = 0
    len1 = len(coefs_list1)
    len2 = len(coefs_list2)
    for i in range(len1 - 1, -1, -1):
        if coefs_list1[i] != 0:
            c1 = i
            break
    for i in range(len2 - 1, -1, -1):
        if coefs_list2[i] != 0:
            c2 = i
            break
    return c1 < c2

In [5]:
def invertmodprime(f,p):
    Zx.<x> = ZZ[]
    Zq.<z> = PolynomialRing(Integers(p))
    ZQphi.<Z> = Zq.quotient(z^N-1)
    a = f % p
    a = a.subs(x=z)
    k = 0
    b = 1*z^0
    c = 0*z^0
    f = a 
    g = z^N-1

    if a.gcd(g) != 1:
        raise Exception("inversion dosen't exist!")      
    while True:
        while list(f)[0] == 0:
            f /= Z
            c *= Z
            k += 1        
        if findDegree(list(f)) == 0:
            b = 1/list(f)[0] * b
            res = Z^(N-k) * b
            return Zx(res.lift())       
        if compareDegrees(list(f), list(g)):
            f, g = g, f
            b, c = c, b       
        u = list(f)[0] * (1/list(g)[0])
        f -= u*g
        b -= u*c

In [6]:
def invertmodpowerof2(f, p):
    deg = int(math.log(p, 2))
    p = 2   
    q = p
    b = invertmodprime(f, p)
    while q < p^deg:
        q = q^2
        b = b * (2 - f*b) % q % (x^N-1)        
    b = b % p^deg % (x^N - 1)
    return b

In [7]:
def validate_params():
    
  
    if q > p and gcd(p,q) == 1:
        return True
    return False

In [8]:
def generate_polynomial(d1, d2, N):

    result = [1]*d1 + [-1]*d2 + [0]*(N-d1-d2)
    shuffle(result)

    return Zx(result)

In [28]:
m=-1+x^3-x^4-x^8+x^9+x^10

In [9]:
def generate_keys(polynomial_1 = None, polynomial_2= None):
    if validate_params():
        while True:
            try:
                if polynomial_1 is None or polynomial_2 is None:   

                    f = generate_polynomial(d+1, d)
                    g = generate_polynomial(d, d)
                else:
                    f = polynomial_1
                    g = polynomial_2

                f_q = invertmodpowerof2(f,q)

                f_p = invertmodprime(f,p)  
                break

            except:
                pass 

        public_key = balancedmod(p * convolution(f_q,g),q)

        secret_key = f,f_p
        return public_key,secret_key

    else:
        print("")

In [10]:
public_key =-16*x^10-13*x^9+12*x^8-13*x^7+15*x^6-8*x^5+12*x^4-12*x^3 - 10*x^2 - 7*x + 8
def encrypt(message, public_key, r):
    return balancedmod(convolution(public_key,r) + message,q)

In [11]:
e=encrypt(-1+x^3-x^4-x^8+x^9+x^10, public_key, -1+x^2+x^3+x^4-x^5-x^7).change_ring(Integers(q))

In [12]:
print(e)

19*x^10 + 6*x^9 + 25*x^8 + 7*x^7 + 30*x^6 + 16*x^5 + 14*x^4 + 24*x^3 + 26*x^2 + 11*x + 14


In [38]:
def decrypt(encrypted_message, secret_key):
    f,f_p = secret_key

    a = balancedmod(convolution(encrypted_message,f),q)

    return balancedmod(convolution(a,f_p),p)

In [39]:
encrypted_message=encrypt(-1+x^3-x^4-x^8+x^9+x^10,public_key,-1+x^2+x^3+x^4-x^5-x^7)
a = convolution(encrypted_message,f)

In [18]:
#The next step is to find b which is $\sage{balancedmod(a,p)}$

25*x^10 + 29*x^9 - 27*x^8 + 7*x^7 + 6*x^6 + 7*x^5 - 22*x^4 - 11*x^3 + 54*x^2 - 7*x - 61


In [40]:
c=decrypt(encrypted_message, (-x^10 + x^9 + x^6 - x^4 + x^2 + x - 1, 2*x^9 + x^8 + 2*x^7 + x^5 + 2*x^4 + 2*x^3 + 2*x + 1))

In [41]:
#and we have the same messages
print(c)

x^10 + x^9 - x^8 - x^4 + x^3 - 1
